# Herschrijven RR-boundaries

Notebook voor het schrijven van de koppeling tussen het RR-model en het FM-model zonder daarbij de afzondelijke modellen te schrijven.

Let op (!): voor het uitvoeren van deze notebook is er een HyDAMO model opgeslagen in Python-pickle nodig. Deze wordt weggeschreven als onderdeel van de modelbouw

In [1]:
from delft3dfmpy import DFlowRRModel, DFlowRRWriter, DFlowFMModel, DFlowFMWriter
from pathlib import Path
import hydrotools
import pandas as pd

drrmodel = DFlowRRModel()
dfmmodel = DFlowFMModel()

In [2]:
data_path = Path(r"..\data").absolute().resolve()
sobek_rr = data_path.joinpath("sobek_rr")

In [3]:
hydamo = hydrotools.load_model(file_name=Path(r"../hydamo_model/boezemmodel_v4.pickle"))
dfmmodel.network.set_branches(hydamo.branches)

In [4]:
sobek_case = "20210826 Boezemmodel 0D1D"
sobek = hydrotools.Sobek(sobek_rr.joinpath("TKI3_NZV.lit"))
rr_laterals = sobek.read_rr_laterals(sobek_case)

In [5]:
for k,v in rr_laterals.items():
    if "branch" in v.keys():
        drrmodel.external_forcings.add_boundary_node(k, v["x"], v["y"])
        dfmmodel.external_forcings.laterals[k] = {
            'branchid': v["branch"],
            'branch_offset': v["chainage"]                    
                    }

In [6]:
for k,v in drrmodel.external_forcings.boundary_nodes.items():
    v["level"] = rr_laterals[k]["level"]

In [7]:
import pandas as pd
from config import dimr_path
start_datetime = pd.Timestamp('2020-01-01 00:00:00')
end_datetime = start_datetime + pd.Timedelta(days=15)

drrmodel.d3b_parameters['Timestepsize'] = 300
drrmodel.d3b_parameters['StartTime'] = start_datetime.strftime('%Y/%m/%d;%H:%M:%S') # should be equal to refdate for D-HYDRO
#drrmodel.d3b_parameters['EndTime'] = end_datetime.strftime('"%Y/%m/%d;%H:%M:%S"')
drrmodel.d3b_parameters['EndTime'] = end_datetime.strftime('%Y/%m/%d;%H:%M:%S')
drrmodel.d3b_parameters['RestartIn'] = 0
drrmodel.d3b_parameters['RestartOut'] = 0
drrmodel.d3b_parameters['RestartFileNamePrefix'] ='Test'
drrmodel.d3b_parameters['UnsaturatedZone'] = 1
drrmodel.d3b_parameters['UnpavedPercolationLikeSobek213']=-1
drrmodel.d3b_parameters['VolumeCheckFactorToCF']=100000
drrmodel.dimr_path = dimr_path


rr_writer = DFlowRRWriter(drrmodel,
                          output_dir=r"modellen\new_boundaries",
                          name="boezemmodel")

rr_writer.copyRRFiles()
    
hydrotools.write_rr_boundaries(rr_writer, write_level=True)
sobek.copy_rr(sobek_case,
              rr_writer.output_dir,
              sobek_rr.joinpath("Sobek_3b.fnm"))

rr_writer.write_coupling()


In [1]:
dfmmodel.mdu_parameters["refdate"] = int(start_datetime.strftime("%Y%m%d"))
dfmmodel.mdu_parameters["tstart"] = 0.0 * 3600
dfmmodel.mdu_parameters["tstop"] = 10 * 24 * 3600
dfmmodel.mdu_parameters["hisinterval"] = "600. 0. 0."
dfmmodel.mdu_parameters["mapinterval"] = "600. 0. 0."
dfmmodel.mdu_parameters["wrirst_bnd"] = 0
dfmmodel.mdu_parameters["cflmax"] = 0.7
dfmmodel.mdu_parameters["outputdir"] = "pir_winter"
dfmmodel.dimr_path = dimr_path

fm_writer = DFlowFMWriter(dfmmodel, output_dir=r"modellen\new_boundaries", name="boezemmodel")
fm_writer.write_boundary_conditions()
fm_writer.write_laterals()

NameError: name 'start_datetime' is not defined